<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.5: Putting it all Together: An FIR Filter
**Prev: [Sequential Logic](2.4_sequential_logic.ipynb)**<br>
**Next: [ChiselTest (was chisel-testers2)](2.6_chiseltest.ipynb)**

## Motivation
Now that you've learned the basics of Chisel, let's use that knowledge to build a FIR (finite impulse response) filter module! FIR filters are very common in digital signal processing applications. Also, the FIR filter will reappear frequently in module 3, so it's important that you don't filter out this module by skipping ahead! If you are unfamiliar with FIR filters, head over to the article on [trusty Wikipedia](https://en.wikipedia.org/wiki/Finite_impulse_response) to learn more.

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# FIR Filter

The FIR filter you will design performs the following operation.

<img src="images/fir.jpg" width="720">

Basically, this does a elementwise multiplication of the element of the filter coefficients with the elements of the input signal and outputs the sum (also called a _convolution_).

Or, a signals definition:

$y[n] = b_0 x[n] + b_1 x[n-1] + b_2 x[n-2] + ...$
 - $y[n]$ is the output signal at time $n$
 - $x[n]$ is the input signal
 - $b_i$ are the filter coefficients or impulse response
 - $n-1$, $n-2$, ... are time $n$ delayed by 1, 2, ... cycles
 
## 8-bit Specification

Build a 4-element FIR filter where the four filter coefficients are parameters. A module skeleton and basic tests are provided for you.
Note that both the input and output are 8-bit unsigned integers. You will need to save necessary state (like delayed signal values) using constructs like shift registers. Use the provided testers to check your implementation.
Registers with constant inputs can be created using a `ShiftRegister` of shift value 1, or by using the `RegNext` construct.

Note: for the tests to pass, your registers must be initialized to `0.U`.

In [ ]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  ???
}

In [ ]:
// Simple sanity check: a element with all zero coefficients should always produce zero
test(new My4ElementFir(0, 0, 0, 0)) { c =>
    c.io.in.poke(0.U)
    c.io.out.expect(0.U)
    c.clock.step(1)
    c.io.in.poke(4.U)
    c.io.out.expect(0.U)
    c.clock.step(1)
    c.io.in.poke(5.U)
    c.io.out.expect(0.U)
    c.clock.step(1)
    c.io.in.poke(2.U)
    c.io.out.expect(0.U)
}

In [ ]:
// Simple 4-point moving average
test(new My4ElementFir(1, 1, 1, 1)) { c =>
    c.io.in.poke(1.U)
    c.io.out.expect(1.U)  // 1, 0, 0, 0
    c.clock.step(1)
    c.io.in.poke(4.U)
    c.io.out.expect(5.U)  // 4, 1, 0, 0
    c.clock.step(1)
    c.io.in.poke(3.U)
    c.io.out.expect(8.U)  // 3, 4, 1, 0
    c.clock.step(1)
    c.io.in.poke(2.U)
    c.io.out.expect(10.U)  // 2, 3, 4, 1
    c.clock.step(1)
    c.io.in.poke(7.U)
    c.io.out.expect(16.U)  // 7, 2, 3, 4
    c.clock.step(1)
    c.io.in.poke(0.U)
    c.io.out.expect(12.U)  // 0, 7, 2, 3
}

In [ ]:
// Nonsymmetric filter
test(new My4ElementFir(1, 2, 3, 4)) { c =>
    c.io.in.poke(1.U)
    c.io.out.expect(1.U)  // 1*1, 0*2, 0*3, 0*4
    c.clock.step(1)
    c.io.in.poke(4.U)
    c.io.out.expect(6.U)  // 4*1, 1*2, 0*3, 0*4
    c.clock.step(1)
    c.io.in.poke(3.U)
    c.io.out.expect(14.U)  // 3*1, 4*2, 1*3, 0*4
    c.clock.step(1)
    c.io.in.poke(2.U)
    c.io.out.expect(24.U)  // 2*1, 3*2, 4*3, 1*4
    c.clock.step(1)
    c.io.in.poke(7.U)
    c.io.out.expect(36.U)  // 7*1, 2*2, 3*3, 4*4
    c.clock.step(1)
    c.io.in.poke(0.U)
    c.io.out.expect(32.U)  // 0*1, 7*2, 2*3, 3*4
}

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + 
    x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + 
    x_n3 * b3.U(8.W)
</pre></article></div></section></div>

---
# FIR Filter Generator

For this module, we'll be using a slightly modified example from [Module 3.2: Generators: Collection](3.2_collections.ipynb).
If you haven't started Module 3.2, don't worry.
You'll learn about the details of how `MyManyDynamicElementVecFir` works, but the basic idea is that it is a FIR filter generator.

The generator has one parameter: length.
That parameter dictates how many taps the filter has, and the taps are inputs to the hardware `Module`.

The generator has 3 inputs:
* in, the input to the filter
* valid, a boolean that says when the input is valid
* consts, a vector for all the taps

and 1 output:
* out, the filtered input

<img src="images/fir.jpg" style="width:450px;"/>

In [ ]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val valid = Input(Bool())
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })
  
  // Such concision! You'll learn what all this means later.
  val taps = Seq(io.in) ++ Seq.fill(io.consts.length - 1)(RegInit(0.U(8.W)))
  taps.zip(taps.tail).foreach { case (a, b) => when (io.valid) { b := a } }

  io.out := taps.zip(io.consts).map { case (a, b) => a * b }.reduce(_ + _)
}

visualize(() => new MyManyDynamicElementVecFir(4))

---
# DspBlock

Integrating DSP components into a larger system can be challenging and error prone.
The [rocket section of the dsptools repository](https://github.com/ucb-bar/dsptools/tree/master/rocket) consists of useful generators that should help with such tasks.

One of the core abstractions is the notion of a `DspBlock`.
A `DspBlock` has:
* AXI-4 Stream input and output
* Memory-mapped status and control (in this example, AXI4)

<img src="images/fir_filter.png" style="width:800px;"/>

`DspBlock`s use diplomatic interfaces from rocket.
[This site](https://www.lowrisc.org/docs/diplomacy/) has a good overview of the basic of diplomacy, but don't worry too much about how it's working for this example.
Diplomacy really shines when you're connecting a lot of different blocks together to form a complex SoC.
In this example, we're just making a single peripheral.
The `StandaloneBlock` traits are mixed in to make diplomatic interfaces work as top-level IOs.
You only need them when the `DspBlock` is being used as a top level interface without any diplomatic connections.

The following code wraps the FIR filter in AXI4 interfaces.


In [ ]:
import dspblocks._
import freechips.rocketchip.amba.axi4._
import freechips.rocketchip.amba.axi4stream._
import freechips.rocketchip.config._
import freechips.rocketchip.diplomacy._
import freechips.rocketchip.regmapper._

//
// Base class for all FIRBlocks.
// This can be extended to make TileLink, AXI4, APB, AHB, etc. flavors of the FIR filter
//
abstract class FIRBlock[D, U, EO, EI, B <: Data](val nFilters: Int, val nTaps: Int)(implicit p: Parameters)
// HasCSR means that the memory interface will be using the RegMapper API to define status and control registers
extends DspBlock[D, U, EO, EI, B] with HasCSR {
    // diplomatic node for the streaming interface
    // identity node means the output and input are parameterized to be the same
    val streamNode = AXI4StreamIdentityNode()
    
    // define the what hardware will be elaborated
    lazy val module = new LazyModuleImp(this) {
        // get streaming input and output wires from diplomatic node
        val (in, _)  = streamNode.in(0)
        val (out, _) = streamNode.out(0)

        require(in.params.n >= nFilters,
                s"""AXI-4 Stream port must be big enough for all 
                   |the filters (need $nFilters,, only have ${in.params.n})""".stripMargin)

        // make registers to store taps
        val taps = Reg(Vec(nFilters, Vec(nTaps, UInt(8.W))))

        // memory map the taps, plus the first address is a read-only field that says how many filter lanes there are
        val mmap = Seq(
            RegField.r(64, nFilters.U, RegFieldDesc("nFilters", "Number of filter lanes"))
        ) ++ taps.flatMap(_.map(t => RegField(8, t, RegFieldDesc("tap", "Tap"))))

        // generate the hardware for the memory interface
        // in this class, regmap is abstract (unimplemented). mixing in something like AXI4HasCSR or TLHasCSR
        // will define regmap for the particular memory interface
        regmap(mmap.zipWithIndex.map({case (r, i) => i * 8 -> Seq(r)}): _*)

        // make the FIR lanes and connect inputs and taps
        val outs = for (i <- 0 until nFilters) yield {
            val fir = Module(new MyManyDynamicElementVecFir(nTaps))
            
            fir.io.in := in.bits.data((i+1)*8, i*8)
            fir.io.valid := in.valid && out.ready
            fir.io.consts := taps(i)            
            fir.io.out
        }

        val output = if (outs.length == 1) {
            outs.head
        } else {
            outs.reduce((x: UInt, y: UInt) => Cat(y, x))
        }

        out.bits.data := output
        in.ready  := out.ready
        out.valid := in.valid
    }
}

// make AXI4 flavor of FIRBlock
abstract class AXI4FIRBlock(nFilters: Int, nTaps: Int)(implicit p: Parameters) extends FIRBlock[AXI4MasterPortParameters, AXI4SlavePortParameters, AXI4EdgeParameters, AXI4EdgeParameters, AXI4Bundle](nFilters, nTaps) with AXI4DspBlock with AXI4HasCSR {
    override val mem = Some(AXI4RegisterNode(
        AddressSet(0x0, 0xffffL), beatBytes = 8
    ))
}

// running the code below will show what firrtl is generated
// note that LazyModules aren't really chisel modules- you need to call ".module" on them when invoking the chisel driver
// also note that AXI4StandaloneBlock is mixed in- if you forget it, you will get weird diplomacy errors because the memory
// interface expects a master and the streaming interface expects to be connected. AXI4StandaloneBlock will add top level IOs
// println(chisel3.Driver.emit(() => LazyModule(new AXI4FIRBlock(1, 8)(Parameters.empty) with AXI4StandaloneBlock).module))

## Testing

Testing `DspBlock`s is a little different.
Now we're dealing with memory interfaces and `LazyModule`s.
dsptools has some features that help test `DspBlock`s.

One important feature is `MemMasterModel`.
The trait defines functions like `memReadWord` and `memWriteWord`- generic functions for generating memory traffic.
This allows you to write one generic test that can be specialized to the memory interface you are using- for example, you write one test and then specialize it for the TileLink and AXI4 interfaces.

The code below tests the `FIRBlock` this way.

In [ ]:
import dsptools.tester.MemMasterModel
import freechips.rocketchip.amba.axi4

abstract class FIRBlockTester[D, U, EO, EI, B <: Data](c: FIRBlock[D, U, EO, EI, B]) extends PeekPokeTester(c.module) with MemMasterModel {
    // check that address 0 is the number of filters
    require(memReadWord(0) == c.nFilters)
    // write 1 to all the taps
    for (i <- 0 until c.nFilters * c.nTaps) {
        memWriteWord(8 + i * 8, 1)
    }
}

// specialize the generic tester for axi4
class AXI4FIRBlockTester(c: AXI4FIRBlock with AXI4StandaloneBlock) extends FIRBlockTester(c) with AXI4MasterModel {
    def memAXI = c.ioMem.get
}

// invoking testers on lazymodules is a little strange.
// note that the firblocktester takes a lazymodule, not a module (it calls .module in "extends PeekPokeTester()").
val lm = LazyModule(new AXI4FIRBlock(1, 8)(Parameters.empty) with AXI4StandaloneBlock)
chisel3.iotesters.Driver(() => lm.module) { _ => new AXI4FIRBlockTester(lm) }

<span style="color:red">**Exercise: TileLink**</span><br>

Add a version of `FIRBlock` that uses TileLink for its memory interconnect, and extend the `FIRBlockTester` to use TileLink.

---
# You're done!

[Return to the top.](#top)